In [20]:
import requests
import pandas as pd

api_key = "2937e2b78f1093a52d383bda8dd05f928b49d4aa"

url = "https://api.scraperlink.com/investpy/"
params = {
    "email": "tfginfotrading@gmail.com",
    "type": "historical_data",
    "product": "stocks",
    "country": "spain",
    "symbol": "ACS",
    "from_date": "04/01/2014",
    "to_date": "04/04/2024"
}
headers = {
    "Authorization": f"Bearer {api_key}"
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    # Hacer algo con los datos devueltos por la API
    # print(data)
    # Crear un DataFrame con los datos obtenidos
    df = pd.DataFrame(data['data'])  # Suponiendo que 'data' es la clave que contiene los datos relevantes
    # Reverse the order of the rows in the DataFrame
    df = df.iloc[::-1]
    # Reset the index of the DataFrame to start from 0
    df = df.reset_index(drop=True)
    # Renombrar las columnas a 'price' y 'fecha'
    df = df.rename(columns={'last_close': 'price', 'rowDate': 'fecha'})

    # Crear una nueva vista del DataFrame con solo las columnas 'price' y 'fecha'
    df_vista = df[['price', 'fecha']]
    # Imprimir el DataFrame resultante
    print(df_vista)
    # Export the DataFrame to an Excel file
    df_vista.to_excel('output.xlsx', index=False)
else:
    print(f"Error al obtener datos: {response.status_code}")



       price         fecha
0     27.181  Apr 01, 2014
1     26.962  Apr 02, 2014
2     27.305  Apr 03, 2014
3     27.775  Apr 04, 2014
4     27.651  Apr 07, 2014
...      ...           ...
2556  40.040  Mar 27, 2024
2557  38.780  Mar 28, 2024
2558  38.480  Apr 02, 2024
2559  38.720  Apr 03, 2024
2560  38.240  Apr 04, 2024

[2561 rows x 2 columns]


In [18]:
import pandas as pd



       price         fecha
0     27.181  Apr 01, 2014
1     26.962  Apr 02, 2014
2     27.305  Apr 03, 2014
3     27.775  Apr 04, 2014
4     27.651  Apr 07, 2014
...      ...           ...
2556  40.040  Mar 27, 2024
2557  38.780  Mar 28, 2024
2558  38.480  Apr 02, 2024
2559  38.720  Apr 03, 2024
2560  38.370  Apr 04, 2024

[2561 rows x 2 columns]


In [ ]:

import csv
from bs4 import BeautifulSoup


In [ ]:
#SOLO LOS PARTIDOS DE LIGA


from bs4 import BeautifulSoup
import pandas as pd

# Lista de archivos HTML a procesar
html_files = [
    'html/2014-2015.html',
    'html/2015-2016.html',
    'html/2016-2017.html',
    'html/2017-2018.html',
    'html/2018-2019.html',
    'html/2019-2020.html',
    'html/2020-2021.html',
    'html/2021-2022.html',
    'html/2022-2023.html'
]

# Lista para almacenar los datos de todos los archivos
data = []

for file_name in html_files:
    with open(file_name, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        matches = soup.find_all('a', class_='match-link', attrs={'data-cy': 'match'})
        
        for match in matches:
            competition_elem = match.find('div', class_='middle-info')
            if competition_elem and 'Primera División' in competition_elem.get_text(strip=True):
                date_elem = match.find('div', class_='date-transform')
                home_team_elem = match.find('div', class_='team-name', itemprop='name')
                away_team_elems = match.find_all('div', class_='team-name', itemprop='name')
                result_elem = match.find('div', class_='marker')
                
                if date_elem and home_team_elem and len(away_team_elems) > 1 and result_elem:
                    result_spans = result_elem.find_all('span')
                    if len(result_spans) == 3:  # Ensure there are two result spans and one dash
                        result_local = result_spans[1].text.strip()
                        result_visitante = result_spans[2].text.strip()
                        data.append([
                            date_elem.text.strip(),
                            competition_elem.text.strip(),
                            home_team_elem.text.strip(),
                            away_team_elems[1].text.strip(),
                            result_elem.text.strip(),
                            result_local,
                            result_visitante
                        ])

# Crear un DataFrame con los datos combinados
df = pd.DataFrame(data, columns=['Fecha', 'Competición', 'Equipo Local', 'Equipo Visitante','Resultado', 'ResultadoLocal', 'ResultadoVisitante'])

# Después de crear el DataFrame
for index, row in df.iterrows():
    if row['Equipo Local'] == 'Real Madrid':
        if int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Ganado'
        elif int(row['ResultadoLocal']) < int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Perdido'
        else:
            df.loc[index, 'Decision'] = 'Empatado'
    else:
        if int(row['ResultadoVisitante']) > int(row['ResultadoLocal']):
            df.loc[index, 'Decision'] = 'Ganado'
        elif int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Perdido'
        else:
            df.loc[index, 'Decision'] = 'Empatado'



# Guardar los datos en un archivo Excel
df.to_excel('real_madrid_primera_division_matches_combined.xlsx', index=False)

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [ ]:
#En directo de la url

import pandas as pd
from bs4 import BeautifulSoup

import requests

url = "https://es.besoccer.com/equipo/partidos/real-madrid"
response = requests.get(url)

# Lista para almacenar los datos de todos los archivos
data = []

if response.status_code == 200:
    respuesta = response.text
    soup = BeautifulSoup(respuesta, 'html.parser')
    matches = soup.find_all('a', class_='match-link', attrs={'data-cy': 'match'})

        
    for match in matches:
            competition_elem = match.find('div', class_='middle-info')
            if competition_elem and 'Primera División' in competition_elem.get_text(strip=True):
                date_elem = match.find('div', class_='date-transform')
                home_team_elem = match.find('div', class_='team-name', itemprop='name')
                away_team_elems = match.find_all('div', class_='team-name', itemprop='name')
                result_elem = match.find('div', class_='marker')
                
                if date_elem and home_team_elem and len(away_team_elems) > 1 and result_elem:
                    result_spans = result_elem.find_all('span')
                    if len(result_spans) == 3:  # Ensure there are two result spans and one dash
                        result_local = result_spans[1].text.strip()
                        result_visitante = result_spans[2].text.strip()
                        data.append([
                            date_elem.text.strip(),
                            competition_elem.text.strip(),
                            home_team_elem.text.strip(),
                            away_team_elems[1].text.strip(),
                            result_elem.text.strip(),
                            result_local,
                            result_visitante
                        ])

# Crear un DataFrame con los datos combinados
    df = pd.DataFrame(data, columns=['Fecha', 'Competición', 'Equipo Local', 'Equipo Visitante','Resultado', 'ResultadoLocal', 'ResultadoVisitante'])

# Después de crear el DataFrame
    for index, row in df.iterrows():
        if row['Equipo Local'] == 'Real Madrid':
            if int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
                df.loc[index, 'Decision'] = 'Ganado'
            elif int(row['ResultadoLocal']) < int(row['ResultadoVisitante']):
                df.loc[index, 'Decision'] = 'Perdido'
            else:
                df.loc[index, 'Decision'] = 'Empatado'
        else:
            if int(row['ResultadoVisitante']) > int(row['ResultadoLocal']):
                df.loc[index, 'Decision'] = 'Ganado'
            elif int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
                df.loc[index, 'Decision'] = 'Perdido'
            else:
                df.loc[index, 'Decision'] = 'Empatado'



# Guardar los datos en un archivo Excel
    df.to_excel('de la url.xlsx', index=False)
    # Aquí puedes procesar los datos obtenidos de la URL
else:
    print("Error al obtener la URL. Código de estado:", response.status_code)



        

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [ ]:
#TODOS LOS PARTIDOS

import pandas as pd
from bs4 import BeautifulSoup

# Lista de archivos HTML a procesar
html_files = [
    'html/2014-2015.html',
    'html/2015-2016.html',
    'html/2016-2017.html',
    'html/2017-2018.html',
    'html/2018-2019.html',
    'html/2019-2020.html',
    'html/2020-2021.html',
    'html/2021-2022.html',
    'html/2022-2023.html'
]

# Lista para almacenar los datos de todos los archivos
data = []

for file_name in html_files:
    with open(file_name, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        matches = soup.find_all('a', class_='match-link', attrs={'data-cy': 'match'})
        
        for match in matches:
                competition_elem = match.find('div', class_='middle-info')
                date_elem = match.find('div', class_='date-transform')
                home_team_elem = match.find('div', class_='team-name', itemprop='name')
                away_team_elems = match.find_all('div', class_='team-name', itemprop='name')
                result_elem = match.find('div', class_='marker')
                
                if date_elem and home_team_elem and len(away_team_elems) > 1 and result_elem:
                    result_spans = result_elem.find_all('span')
                    if len(result_spans) == 3:  # Ensure there are two result spans and one dash
                        result_local = result_spans[1].text.strip()
                        result_visitante = result_spans[2].text.strip()
                        data.append([
                            date_elem.text.strip(),
                            competition_elem.text.strip(),
                            home_team_elem.text.strip(),
                            away_team_elems[1].text.strip(),
                            result_elem.text.strip(),
                            result_local,
                            result_visitante
                        ])

# Crear un DataFrame con los datos combinados
df = pd.DataFrame(data, columns=['Fecha', 'Competición', 'Equipo Local', 'Equipo Visitante','Resultado', 'ResultadoLocal', 'ResultadoVisitante'])

# Después de crear el DataFrame
for index, row in df.iterrows():
    if row['Equipo Local'] == 'Real Madrid':
        if int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Ganado'
        elif int(row['ResultadoLocal']) < int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Perdido'
        else:
            df.loc[index, 'Decision'] = 'Empatado'
    else:
        if int(row['ResultadoVisitante']) > int(row['ResultadoLocal']):
            df.loc[index, 'Decision'] = 'Ganado'
        elif int(row['ResultadoLocal']) > int(row['ResultadoVisitante']):
            df.loc[index, 'Decision'] = 'Perdido'
        else:
            df.loc[index, 'Decision'] = 'Empatado'



# Guardar los datos en un archivo Excel
df.to_excel('real_madrid_matches_combined.xlsx', index=False)